In [10]:
from langchain_groq import ChatGroq
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
# from langchain_huggingface import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import ServerlessSpec
from pinecone.grpc import PineconeGRPC as pinecone

In [11]:
def pdf_loader(path):
    loader = DirectoryLoader(path, glob="*.pdf", loader_cls= PyPDFLoader)
    pages = loader.load()
    return pages

In [12]:
pages = pdf_loader(r"data/")

In [13]:
def text_splitter(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    text_chunks = text_splitter.split_documents(text)
    return text_chunks

In [14]:
text_chunks = text_splitter(pages)
len(text_chunks)

7079

In [16]:
def get_embedding():
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
  return embedding

In [17]:
embedding = get_embedding()
query_result = embedding.embed_query("Hello, how are you?")
len(query_result)


C:\Users\SAMANWAYA\AppData\Local\Temp\ipykernel_11620\952409544.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')


384

In [19]:
from dotenv import load_dotenv
import os

load_dotenv()

PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

In [22]:
pc = pinecone(PINECONE_API_KEY)
index_name = "medicalchatbot"
pc.create_index(name=index_name,
                dimension=384,
                metric="cosine",
                spec=ServerlessSpec(cloud="aws", region="us-east-1"))

In [20]:
from langchain_pinecone import PineconeVectorStore

In [23]:
docserach = PineconeVectorStore.from_documents(documents=text_chunks, embedding=embedding, index_name="medicalchatbot")


In [21]:
docs = PineconeVectorStore.from_existing_index(index_name="medicalchatbot", embedding=embedding)
docs

In [22]:
retriver = docs.as_retriever(search_type="similarity", search_kwargs={"k": 3})
output = retriver.invoke("What is Headache?")
output

[Document(id='545534f3-b670-4358-8904-6e6403323495', metadata={'page': 213.0, 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='occur over the course of a day.\nMigraine headache—An intense throbbing pain\nthat occurs on one or both sides of the head. The\nheadache is usually accompanied by other symp-\ntoms, such as nausea, vomiting, and aversion to\nlight.\nProphylactic—Referring to treatment that prevents\nsymptoms from occurring.\nTension-type headache—A dull pain that seems to\nexert pressure on the head; the most common\nform of headache.\nsumatriptan injection, and application of local anesthet-'),
 Document(id='f201d281-cb20-40cd-a7bf-14c644233021', metadata={'page': 213.0, 'source': 'data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf'}, page_content='occur over the course of a day.\nMigraine headache—An intense throbbing pain\nthat occurs on one or both sides of the head. The\nheadache is usually accompanied by other symp-\ntoms, such as nausea, vo

In [23]:
import os
GROQ_API_KEY= os.getenv("GROQ_API_KEY")
os.environ["GROQ_API_KEY"] = GROQ_API_KEY
llm = ChatGroq(temperature=0.6, max_tokens=500)

In [24]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate

In [25]:
system_prompt = (
"You are an expert medical assistat of qestion-answering tasks."
"Use the following pieces of retrieved context to answer "
"the question. If you don't find any related context then say that you "
"don't know. Do not give any halusinating answer of this. Use the three sentece maximum and keep the "
"answer concise."
"\n\n"
"{context}"
 )

chat_prompt = ChatPromptTemplate.from_messages([
  ("system", system_prompt),
  ("user", "{input}" )]
)

In [132]:
stuff_chain =create_stuff_documents_chain(llm, chat_prompt)
retriver_chain = create_retrieval_chain(retriver, stuff_chain)
question = "What is Dental trauma?"
response_dict = retriver_chain.invoke({"input" : question})
response = response_dict["answer"] if isinstance(response_dict, dict) else str(response_dict)
# print(response["answer"])

In [133]:
from langchain_core.messages import HumanMessage, AIMessage
chat_history = []
chat_history.extend([
  HumanMessage(content=question),
  AIMessage(content=response)
])

In [134]:
chat_history

[HumanMessage(content='What is Dental trauma?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Dental trauma is injury to the mouth, including teeth, lips, gums, tongue, and jawbones, often caused by contact sports, accidents, or mishaps. It typically results in painful oral tissue damage and requires prompt treatment by a dentist.', additional_kwargs={}, response_metadata={})]

In [136]:
from langchain_core.prompts import MessagesPlaceholder
contextualize_system_prompt = (
  "Given a chat history and latest user question "
  "which might reference context in the chat history, "
  "formulates a standalone question which can be understood "
  "without the chat history. Do not answer the question, "
  "just reformulate it if needed otherwise retuen as it is"
)

contextualize_prompt = ChatPromptTemplate.from_messages([
  ("system", contextualize_system_prompt),
  MessagesPlaceholder(variable_name="chat_history"),
  ("user", "{input}"),
  ("system", "Context: {context}")
])

In [137]:

from langchain.chains import create_history_aware_retriever

create_history_retrieve_chain = create_history_aware_retriever(llm, retriver, contextualize_prompt)
# create_history_retrieve_chain.invoke({"input" :"What is the treatment of this?", "chat_history" : chat_history})


In [139]:
history_stuff_chain = create_stuff_documents_chain(llm, contextualize_prompt)
history_retrieval_chain = create_retrieval_chain(create_history_retrieve_chain, history_stuff_chain)
ans = history_retrieval_chain.invoke({"input" :"What is the Prevention for this?", "chat_history" : chat_history, "context": ""})
print(ans['answer'])

How can dental trauma be prevented?

Preventing dental trauma involves monitoring for potential hazards, taking child-proofing measures, and adopting safety habits. This includes wearing seat belts and securing young children in appropriate car seats, removing tripping and slipping hazards at home, and taking extra precautions for toddlers. Additionally, electrical cords should be tucked away to prevent young children from gnawing on them and receiving severe oral burns.
